In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [736 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,569 kB]
Get:13 http://security.ubuntu.

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"


spark.sparkContext.addFile(url)
sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True, quote="\"", escape="\"")
sales_df.head()

Row(id='f8a53099-ba1c-47d6-9c31-7398aa8f6089', date=datetime.date(2022, 4, 8), date_built=2016, price=936923, bedrooms=4, bathrooms=3, sqft_living=3167, sqft_lot=11733, floors=2, waterfront=1, view=76)

In [4]:
# 2. Create a temporary view of the DataFrame.
sales_df.createOrReplaceTempView('home_sales')


In [6]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
spark.sql("""SELECT round(avg(price),2) AS averagePrice FROM home_sales WHERE bedrooms=4""").show()

+------------+
|averagePrice|
+------------+
|   299661.01|
+------------+



In [7]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql("""SELECT date_built, round(avg(price),2) AS avgPrice FROM home_sales WHERE bedrooms=3 AND bathrooms=3 GROUP BY date_built ORDER BY date_built""").show()


+----------+---------+
|date_built| avgPrice|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [8]:
# 5. What is the average price of a home for each year the home was built that have 3 bedrooms, 3 bathrooms, with two floors and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql("""SELECT date_built, round(avg(price),2) AS avgPrice FROM home_sales WHERE bedrooms=3 AND bathrooms=3 AND floors=2 AND sqft_lot >= 2000 GROUP BY date_built ORDER BY date_built""").show()


+----------+---------+
|date_built| avgPrice|
+----------+---------+
|      2010|280447.23|
|      2011|281413.45|
|      2012|295858.68|
|      2013|295142.13|
|      2014|294195.13|
|      2015|291788.36|
|      2016| 287812.6|
|      2017|282026.59|
+----------+---------+



In [10]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()
spark.sql("""SELECT view, round(avg(price),2) AS avgPrice FROM home_sales WHERE price >= 350000 GROUP BY view ORDER BY view DESC""").show()
print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view|  avgPrice|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 1.0173349380493164 seconds ---


In [11]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [12]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [14]:
# 9. Using the cached data, run the last query above, that calculates the average price of a home per "view" rating, rounded to two decimal places having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()
spark.sql("""SELECT view, round(avg(price),2) AS avgPrice FROM home_sales WHERE price >= 350000 GROUP BY view ORDER BY view DESC""").show()
print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view|  avgPrice|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 0.5857725143432617 seconds ---


In [18]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
sales_df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales")

In [19]:
# 11. Read the parquet formatted data.
p_sales_df=spark.read.parquet('home_sales')

In [20]:
# 12. Create a temporary table for the parquet data.
p_sales_df.createOrReplaceTempView('p_home_sales')

In [21]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()
spark.sql("""SELECT view, round(avg(price),2) AS avgPrice FROM p_home_sales WHERE price >= 350000 GROUP BY view ORDER BY view DESC""").show()
print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view|  avgPrice|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 0.9416601657867432 seconds ---


In [22]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [24]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False